In [19]:
import requests
import pandas as pd
import string
import spacy
import jaro
import re

In [20]:
def get_sheet(src):
    raw_html = requests.get(src).text
    table_html = raw_html[raw_html.find("<tbody>")+7:raw_html.find("</tbody>")]
    table_rows = table_html.split("</tr>")
    grid = []
    for row in table_rows:
        row = row[row.find("</th>")+5:]
        cols = row.split("</td>")
        clean_row = []
        for col in cols:
            col = col[col.find(">")+1:]
            clean_row.append(col)
        grid.append(clean_row)
    return grid

In [21]:
def sheet_to_df(src):
    sheet = get_sheet(src)

    #A dataframe starts when a header is found
    #A header is a the first instance of a set of contiguous columns in the same row that contain at least some letters
    #A dataframe ends when a blank row is found or an empty column is found

    num = 0 #The number of the dataframe
    headers = [] #Each header is a tuple (num, row, col_start, col_end)
    binding_headers = []
    dfs = [] #Each df is a tuple (num, df)

    #First pass: get all the headers
    for row in range(len(sheet)):

        #if every cell in the row is empty, skip row
        if all([sheet[row][col].strip() == "" for col in range(len(sheet[row]))]):
            headers += binding_headers
            binding_headers = []
            continue

        for col in range(len(sheet[row])):

            #Check if the cell is bounded by a header
            if any([col >= header[2] and col <= header[3] for header in binding_headers]):
                continue

            #Check if the cell is commented out
            if sheet[row][col].strip().startswith("//"):
                continue
            
            if re.search('[a-zA-Z]', sheet[row][col]):
                head_start = col
                head_end = col
                while head_end < len(sheet[row]) and re.search('[a-zA-Z]', sheet[row][head_end]):
                    head_end += 1
                binding_headers.append([num, row, head_start, head_end])
                num += 1   
    headers += binding_headers

    #Second pass: get all the dataframes
    for header in headers:
        df = []
        for row in range(header[1], len(sheet)):
            if all([sheet[row][col].strip() == "" for col in range(header[2], header[3])]):
                break
            df_row = []
            for col in range(header[2], header[3]):
                df_row.append(sheet[row][col])
            df.append(df_row)
        cols = df[0]
        data = df[1:]
        df = pd.DataFrame(data, columns=cols)
        dfs.append(df)
    
    return dfs

dfs = sheet_to_df("https://docs.google.com/spreadsheets/d/1SJngKcNkzHupfVBSLLAwyZ5bzHILNRggxsjfXyLMI98/edit?usp=sharing")
dfs

[     X1    Y1    X2    Y2     DIST DELAY (# of cycles)
 0  -128  -128  -128  -128        0                   1
 1  -128  -128   127   127    2.125                   1
 2  -128  -128     0     0      649                   1
 3  -128  -128   127  -128        1                   1
 4   -96   -96  -128   127  108.875                   1
 5     0     0   127   127   646.75                   1
 6   -63   -63    63    63  644.625                   1
 7   -10    10    10   -10    68.75                   1
 8    96   -96    63    63  259.125                   1
 9    33   -33    50    50  206.875                   1,
   Apples Oranges Avocados
 0      1       2        3
 1      4       5        6,
    S  A  B
 0  0  1  2
 1  1  1  3
 2  2  3  2]

In [22]:
def select_df(prompt, dfs):

    #Simplify the prompt to just the nouns
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(prompt)
    nouns = [token.text for token in doc if (token.pos_ == 'NOUN' or token.pos_ == 'PROPN')]
    nouns = [word.lower() for word in nouns] #lowercase all the words (done after finding the nouns to preserve proper nouns)

    all_columns = [df.columns for df in dfs]

    #Find the similarity between the prompt subject and each df
    similarities = []
    for columns in all_columns:
        #Preprocess the columns
        cleaned_columns = [col.lower().strip().translate(str.maketrans('', '', string.punctuation)).split() for col in columns]
        cleaned_columns = [word for col in cleaned_columns for word in col] #flatten the list

        #Find the similarity
        similarity = 0
        for col in cleaned_columns:
            for word in nouns:
                similarity = max(jaro.jaro_winkler_metric(col, word), similarity)

        similarities.append(similarity)

    #Return the df with the highest similarity
    return dfs[similarities.index(max(similarities))]

prompt = "What is the median number of A"
df = select_df(prompt, dfs)
df

,S,A,B
0,0,1,2
1,1,1,3
2,2,3,2
